# Python Basics Review

**Syntax**

<br>Python reads code line by line and uses key words, characters and indenting to store data and execute functions.
<br>Import the Python libraries used in a script at the top.  Then define global variables and functions.
<br>Use comments to document what each section of code is doing.

# Data Types 

**Numbers** - Integer, floating point, complex numbers

**Boolean** - True or False

**Strings** - text

**Lists** - container for data/objects

**Tuples** - container for data/objects

**Dictionaries** - container for data/objects, like a row in a spreadsheet

**Sets** - container for data/objects. Each element in a set is unique, no duplicates

**Arcpy Feature Layer** - A temporary version of a shapefile, feature class or table that allows for spatial and attribute selections.  Any edits to the Feature Layer will be reflected in the source shapefile, feature class or table.

![title](images/DataTypes.jpg)


#### Reference Guide for Data Types:
https://www.geeksforgeeks.org/python-data-types/

# Programming Concepts

* Creating variables!
* Comparison Operators!
* Looping!
* Conditional Statements!
* Python built in functions!
* Defining custom functions!

Reference Guide for Built in Functions:
https://www.geeksforgeeks.org/python-built-in-functions/

Reference guide for Builtin in String Methods:
https://www.w3schools.com/python/python_ref_string.asp

**Bonus - List Comprehension**

List comprehensions are incredibly useful for creating and modifying lists.  It can save you all sorts of space in your code by reducing the number of lines needed to create a new list or filter an existing list.  There are also dict comprehensions!

https://www.w3schools.com/python/python_lists_comprehension.asp

# Process Automation with arcpy
Let's look at an example of a typical workflow in which you may want to use python.  Here we have a series of zipped shapefiles. We would like all of the point layers to contain neighborhood name information. To accomplish this, we'll have to:

- extract each zipped shapefile
- Get a list of all the shapefiles
- Isolate the point shapefiles
- Run a spatial join between the SF Neighborhoods shapefile and the points layers

In [ ]:
import os
import arcpy
import zipfile

In [ ]:
#remember "." stands for the current working directory

# Set an output location for the data
example_folder = r".\Example_Data"
if not os.path.exists(example_folder):
    os.mkdir(example_folder)

In [ ]:
# create a folder for extracted shapefiles
extracted_folder = r".\Example_Data\Extracted_SHP"
if not os.path.exists(extracted_folder):
    os.mkdir(extracted_folder)

In [ ]:
# Unzip all the zipped shapefiles
zips = [f for f in os.listdir('.') if f.endswith('.zip')]
for zf in zips:
    zfile = zipfile.ZipFile(zf)
    zfile.extractall(extracted_folder)

In [ ]:
#Set location for output of geoprocessing tools
output_folder = r".\Example_Data\Output_Folder"
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
# iterate through shapefiles and create clipped shapefiles
arcpy.env.workspace = extracted_folder
shp_list = arcpy.ListFeatureClasses()
print(shp_list)

https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/listfeatureclasses.htm

In [ ]:
shp_list = arcpy.ListFeatureClasses(feature_type = "Point")
print(shp_list)

In [ ]:
for shp in shp_list:
    print('Updating', shp)
    output_shp = os.path.join(output_folder, shp[:-4] + '_spjoin.shp')
    #arcpy.analysis.SpatialJoin(target_features, join_features, out_feature_class)
    arcpy.analysis.SpatialJoin(shp, 'SF_Neighborhoods.shp', output_shp, match_option="COMPLETELY WITHIN")

https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/spatial-join.htm

In [ ]:
arcpy.env.overwriteOutput = True

** BONUS **
#### Putting the above cells all together to review our script
#### Suggestions for improvements on the script?

In [ ]:
import os
import arcpy
import zipfile

# Set an output location for the data
example_folder = r".\Example_Data"
if not os.path.exists(example_folder):
    os.mkdir(example_folder)

# create a folder for extracted shapefiles
extracted_folder = r".\Example_Data\Extracted_SHP"
if not os.path.exists(extracted_folder):
    os.mkdir(extracted_folder)

# Unzip all the zipped shapefiles
zips = [f for f in os.listdir('.') if f.endswith('.zip')]
for zf in zips:
    zfile = zipfile.ZipFile(zf)
    zfile.extractall(extracted_folder)

#Set location for output of geoprocessing tools
output_folder = r".\Example_Data\Output_Folder"
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

# iterate through shapefiles and create clipped shapefiles
arcpy.env.workspace = extracted_folder
shp_list = arcpy.ListFeatureClasses(feature_type = "Point")

for shp in shp_list:
    print('Updating', shp)
    output_shp = os.path.join(output_folder, f'{shp[:-4]}_spjoin.shp')
    arcpy.analysis.SpatialJoin(shp, 'SF_Neighborhoods.shp', output_shp, match_option="COMPLETELY WITHIN")

# Cursors

Arcpy has 3 types of Cursors, Search, Update and Insert.  These can be used to iterate through the table of a shapefile/feature class/feature table and access the values of each row in the table.

Cursors require two things:
an input shapefile/feature class/feature table
<br>A list of columns

This returns an iterator object that can be looped through like a list.

- [arcpy.da.SearchCursor()](https://pro.arcgis.com/en/pro-app/arcpy/data-access/searchcursor-class.htm)
- [arcpy.da.InsertCursor()](https://pro.arcgis.com/en/pro-app/arcpy/data-access/insertcursor-class.htm)
- [arcpy.da.UpdateCursor()](https://pro.arcgis.com/en/pro-app/arcpy/data-access/updatecursor-class.htm)

In [ ]:
extracted_folder = r'C:\Users\eric.samson\Documents\Python\BayGeo_IntermediateClass\Spring2022_BayGeo_IntermediatePython_Workshop1\Example_Data\Extracted_SHP'

In [ ]:
extracted_folder = r'.\Example_Data\Extracted_SHP'

In [ ]:
#Example of creating a Search Cursor, note that da.SearchCursor is the newest version of this

shp = os.path.join(extracted_folder, "SF_Streets.shp")
flds = ['streetname', 'active', 'classcode', 'SHAPE@LENGTH']

cursor = arcpy.da.SearchCursor(shp, flds)

In [ ]:
cursor

In [ ]:
for row in cursor:
    print(type(row), row)

In [ ]:
type(cursor)

In [ ]:
row[3]

In [ ]:
for row in cursor:
    if row[1] == 0:
        print(row)

In [ ]:
#Once the iterator has been iterated through, it needs to be reset before you can iterate again
#Using the reset method
cursor.reset()

In [ ]:
#Now we can iterate again!
for row in cursor:
    if row[1] == 0:
        print(row)

In [ ]:
row

In [ ]:
#The cursors are stored in the RAM memory, if you are working with large tables, you may want to delete them after you're done.
del row
del cursor

In [ ]:
cursor

#### Instead of creating an instance of a cursor and having to delete it, we can use "with"
#### "with" will create a temporary cursor and you will not have to delete it. It will exist within the indented block


In [ ]:
with arcpy.da.SearchCursor(shp, flds) as cursor:
    for row in cursor:
        print(type(row), row)

#### The update cursor has a list data type, tuples are immutable but lists we can adjust

#### Lets update the last row to mixed case using the title string method and the updateRow Cursor method:

In [ ]:
with arcpy.da.UpdateCursor(shp, flds) as cursor:
    for row in cursor:
        if row[0] == 'IRONWOOD WAY':
            row[0] = row[0].title()
        
        cursor.updateRow(row)

In [ ]:
row

#### We can use a search cursor to get all of the unique values within a column:

In [ ]:
#We can use a search cursor to get all of the unique terms within a column:

sf_businesses_sjoin = os.path.join(output_folder, 'SF_Businesses_spjoin.shp')

nhoods = []
with arcpy.da.SearchCursor(sf_businesses_sjoin, 'nhood') as nhood_cursor:
    for row in nhood_cursor:
        nhoods.append(row[0])

nhoods_unique = set(nhoods)

In [ ]:
nhoods_unique

#### Doing it in a comprehension!

In [ ]:
unique_values_set = set(row[0] for row in arcpy.da.SearchCursor(sf_businesses_sjoin, "nhood"))

#### Even better! Comprehension inside a function that we can continue to use whenever we need it!

In [ ]:
def get_unique_values_of_column(fc, column):
    return set(row[0] for row in arcpy.da.SearchCursor(fc, column))

get_unique_values_of_column(sf_businesses_sjoin, "nhood")

#### Okay, cool. Search Cursors are good at retrieving information from a feature layer, but more commonly I want to update and or fix the information in a feature layer.
#### What's a practical example of using this??

In [ ]:
with arcpy.da.UpdateCursor(sf_businesses_sjoin, ["neighborho", "nhood"]) as cursor:
    for row in cursor:
        if row[0] != row[1]: #row[0] = original neighborhood field
            row[0] = row[1]  #row[1] = new neighborhood field
        cursor.updateRow(row)

### In Summary:

If you find yourself using selectbyattributes and calculate fields over and over again in your script, 
you probably should be using update cursors instead!

Update cursors can be 10x faster then running a selection and then using the calculate fields tool!

# Memory Workspace

The memory workspace allows you to create temporary spatial layers that exist in the RAM memory.  
This can make scripts run faster and also avoid creating shapefiles/feature classes for every step of a script.  
Once the script or Python session ends, the layers will no longer exist.

Not all tools will work with the in memory workspace, so you may need to adjust your methods in some cases.  
It's also not possible to create folders within the in memory workspace.

"in_memory" is the temporary workspace for ArcMap, "memory" is the new version used in ArcPro.
Keep this in mind if you are developing tools people will use with ArcMap.  The "in_memory" workspace also doesn't support subtypes or domains.  

These temporary workspaces are great for storing intermediary layers, but if you are working with large datasets, 
you may want to delete them once they are no longer needed to free up space in your RAM.

Example of format for creating a temporary buffer feature

**ArcPro** - r"memory\Buffer"

**ArcMap** - r"in_memory\Buffer"


In [ ]:
#Let's use the memory workspace to save a temporary layer in some geoprocessing
#We can use the over write output setting to overwrite the temporary layer

arcpy.env.overwriteOutput = True

locations_of_interest = os.path.join(output_folder, "Locations_of_Interest_spjoin.shp")
locations_buffer = r"memory\locations_buffer"

arcpy.Buffer_analysis(locations_of_interest, locations_buffer, "1 Mile")

arcpy.CopyFeatures_management(locations_buffer, "permanent_locations_buffer.shp")

In [ ]:
#We can delete the temporary layer when we don't need it anymore
#Only really needed if you are using large datasets and will continue in your Python session/script
arcpy.Delete_management(locations_buffer)